In [2]:
import requests
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
from random import randint
from time import sleep

In [20]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"}
url = 'https://www.vivino.com/'

# Get Cache key to get country codes and type of wines
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
script = soup.find('script', text = re.compile('var vivinoCacheKey'))
vivinoCacheKey = str(script).split('vivinoCacheKey = ')[-1].split(';')[0].replace("'",'').strip()


rows = []
# Iterate through countries and wine types
api_url = 'https://www.vivino.com/api/explore/explore'
for country in ['ar', 'pt', 'cl', 'at', 'au', 'es', 'de', 'us', 'it', 'fr']:
    payload = {
    "country_code": country.upper(),
    "currency_code":"USD",
    "grape_filter":"varietal",
    "min_rating":"1",
    "min_ratings_count":"200",
    "order_by":"ratings_count",
    "order":"desc",
    "page": '1',
    "price_range_max": "150",
    "price_range_min": "50",
    "wine_type_id": "1"}

    try:
        jsonData = requests.get(api_url, params=payload, headers=headers).json()
        total_pages = math.ceil(jsonData['explore_vintage']['records_matched'] / 100)
        #print('%s' %(country['code'].upper()))
        
        for page in range(1,total_pages+1):
            if page != 1:   
                payload.update({'page':page})
            jsonData = requests.get(api_url, params=payload, headers=headers).json()
            for each in jsonData['explore_vintage']['records']:
                name = f'{each["vintage"]["wine"]["name"]} {each["vintage"]["year"]}'
                year = each["vintage"]["year"]
                id = each["vintage"]["wine"]["id"]
                rating =  each['vintage']['statistics']['ratings_average']
                price = each['price']['amount']
                winery = each["vintage"]["wine"]["winery"]["name"]
                ratings_count = each["vintage"]["statistics"]["ratings_count"]
                country = country
                acidity = each['vintage']['wine']['taste']['structure']['acidity']
                calculated_structure_count = each['vintage']['wine']['taste']['structure']['calculated_structure_count']
                intensity = each['vintage']['wine']['taste']['structure']['intensity']
                sweetness = each['vintage']['wine']['taste']['structure']['sweetness']
                tannin = each['vintage']['wine']['taste']['structure']['tannin'],
                user_structure_count = each['vintage']['wine']['taste']['structure']['user_structure_count'],
                flavor_1 = each['vintage']['wine']['taste']['flavor'][0]['group'],
                flavor_1_count = each['vintage']['wine']['taste']['flavor'][0]['stats']['count'],
                flavor_2 = each['vintage']['wine']['taste']['flavor'][1]['group'],
                flavor_2_count = each['vintage']['wine']['taste']['flavor'][1]['stats']['count'],
                flavor_3 = each['vintage']['wine']['taste']['flavor'][2]['group'],
                flavor_3_count = each['vintage']['wine']['taste']['flavor'][2]['stats']['count'],
                flavor_4 = each['vintage']['wine']['taste']['flavor'][3]['group'],
                flavor_4_count = each['vintage']['wine']['taste']['flavor'][3]['stats']['count'],
                flavor_5 = each['vintage']['wine']['taste']['flavor'][4]['group'],
                flavor_5_count = each['vintage']['wine']['taste']['flavor'][4]['stats']['count'],


                grapes_in_wine = each["vintage"]["wine"]["style"]["grapes"]    #Grape(s)

                
                
                row = {'name':name, "year":year, "wine ID":id, 'rating':rating, 'price':price, 'winery': winery, 'ratings_count': ratings_count, 'country': country,\
                       'acidity': acidity, 'calculated_structure_count': calculated_structure_count, 'intensity': intensity, 'sweetness': sweetness, 'tannin': tannin, 'user_structure_count': user_structure_count,\
                        'flavor_1': flavor_1, 'flavor_1_count': flavor_1_count, 'flavor_2': flavor_2, 'flavor_2_count': flavor_2_count, 'flavor_3': flavor_3, 'flavor_3_count': flavor_3_count, 'flavor_4': flavor_4, 'flavor_4_count': flavor_4_count, 'flavor_5': flavor_5, 'flavor_5_count': flavor_5_count,\
                        'grapes_in_wine': grapes_in_wine}

                rows.append(row)
            print('Aquired page: %s - %s ' %(country, page))
    except:
        continue

df = pd.DataFrame(rows)
df

Aquired page: cl - 1 
Aquired page: au - 1 


,name,year,wine ID,rating,price,winery,ratings_count,country,acidity,calculated_structure_count,...,flavor_1_count,flavor_2,flavor_2_count,flavor_3,flavor_3_count,flavor_4,flavor_4_count,flavor_5,flavor_5_count,grapes_in_wine
0,Malbec Alta 2018,2018,68824,4.6,64.880767,Angélica Zapata,5006,ar,2.571469,792,...,"(658,)","(black_fruit,)","(503,)","(non_oak,)","(384,)","(spices,)","(372,)","(earth,)","(308,)","[{'id': 9, 'name': 'Malbec', 'seo_name': 'malb..."
1,Nicola Catena Parcela Bonarda 2018,2018,5095201,4.6,93.500394,Catena Zapata,2821,ar,3.573292,104,...,"(111,)","(black_fruit,)","(93,)","(spices,)","(80,)","(non_oak,)","(78,)","(red_fruit,)","(59,)","[{'id': 74, 'name': 'Bonarda', 'seo_name': 'bo..."
2,Malbec Argentino 2018,2018,1869,4.7,133.366021,Catena Zapata,2626,ar,2.683492,1249,...,"(756,)","(black_fruit,)","(702,)","(earth,)","(492,)","(non_oak,)","(462,)","(spices,)","(379,)","[{'id': 9, 'name': 'Malbec', 'seo_name': 'malb..."
3,Bramare Malbec Lujan de Cuyo 2019,2019,43974,4.5,61.997177,Viña Cobos,1918,ar,2.609345,850,...,"(650,)","(black_fruit,)","(508,)","(non_oak,)","(427,)","(spices,)","(382,)","(earth,)","(331,)","[{'id': 9, 'name': 'Malbec', 'seo_name': 'malb..."
4,Gran Enemigo 2016,2016,1235058,4.6,72.990863,El Enemigo,1886,ar,3.611121,312,...,"(298,)","(black_fruit,)","(235,)","(earth,)","(192,)","(spices,)","(186,)","(non_oak,)","(184,)","[{'id': 2, 'name': 'Cabernet Sauvignon', 'seo_..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,Brut Rosé Champagne N.V.,N.V.,1211816,4.4,65.849874,Billecart-Salmon,21473,fr,4.220401,518,...,"(1315,)","(citrus_fruit,)","(761,)","(microbio,)","(683,)","(non_oak,)","(602,)","(tree_fruit,)","(601,)","[{'id': 5, 'name': 'Chardonnay', 'seo_name': '..."
145,Brut Rosé Champagne N.V.,N.V.,23051,4.3,59.134590,Veuve Clicquot,20821,fr,4.314180,611,...,"(1049,)","(citrus_fruit,)","(530,)","(microbio,)","(485,)","(non_oak,)","(430,)","(oak,)","(348,)","[{'id': 5, 'name': 'Chardonnay', 'seo_name': '..."
146,Brut Rosé Champagne N.V.,N.V.,1148298,4.3,75.171089,Ruinart,19448,fr,4.169445,418,...,"(835,)","(microbio,)","(396,)","(non_oak,)","(395,)","(citrus_fruit,)","(381,)","(oak,)","(323,)","[{'id': 5, 'name': 'Chardonnay', 'seo_name': '..."
147,Ice Impérial (Demi-Sec) Champagne N.V.,N.V.,1161252,4.2,50.013831,Moët & Chandon,15881,fr,3.924822,279,...,"(293,)","(citrus_fruit,)","(234,)","(spices,)","(181,)","(red_fruit,)","(147,)","(earth,)","(104,)","[{'id': 5, 'name': 'Chardonnay', 'seo_name': '..."


In [32]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0"}
url = 'https://www.vivino.com/'

# Get Cache key to get country codes and type of wines
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
script = soup.find('script', text = re.compile('var vivinoCacheKey'))
vivinoCacheKey = str(script).split('vivinoCacheKey = ')[-1].split(';')[0].replace("'",'').strip()


rows = []
# Iterate through countries and wine types
api_url = 'https://www.vivino.com/api/explore/explore'
for country in ['ar', 'pt', 'cl', 'at', 'au', 'es', 'de', 'us', 'it', 'fr']:
    payload = {
    "country_code": country.upper(),
    "currency_code":"USD",
    "grape_filter":"varietal",
    "min_rating":"1",
    "min_ratings_count":200,
    "order_by":"ratings_count",
    "order":"desc",
    "page": '1',
    "price_range_max": 150,
    "price_range_min": 50,
    "wine_type_id": 1}

    jsonData = requests.get(api_url, params=payload, headers=headers).json()

    df1 = pd.json_normalize(jsonData['explore_vintage']['matches'])
    df2 = pd.json_normalize(jsonData['explore_vintage']['records'])

df1.head()


,prices,vintage.id,vintage.seo_name,vintage.name,vintage.statistics.status,vintage.statistics.ratings_count,vintage.statistics.ratings_average,vintage.statistics.labels_count,vintage.statistics.wine_ratings_count,vintage.statistics.wine_ratings_average,...,price.bottle_type.short_name,price.bottle_type.short_name_plural,price.bottle_type.volume_ml,vintage.wine.winery.background_image.location,vintage.wine.winery.background_image.variations.large,vintage.wine.winery.background_image.variations.medium,vintage.wine.winery.background_image.variations.small,vintage.wine.taste.structure,vintage.wine.style.background_image,vintage.wine.region.background_image
0,"[{'id': 25594303, 'amount': 70.10956937990868,...",1469472,laurent-perrier-brut-cuvee-champagne-rose-nv,Laurent-Perrier Brut Cuvée Champagne Rosé N.V.,Normal,35542,4.4,262835,35542,4.4,...,bottle,bottles,750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[{'id': 21533596, 'amount': 52.01839386443545,...",1782659,ruinart-brut-r-de-ruinart-champagne-nv,Ruinart Brut (R de Ruinart) Champagne N.V.,Normal,22102,4.3,96246,22102,4.3,...,bottle,bottles,750,//images.vivino.com/wineries/backgrounds/3KO1O...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,NaN,NaN,NaN
2,"[{'id': 8980533, 'amount': 65.84987431393853, ...",164942610,champagne-billecart-salmon-brut-rose-champagne-nv,Billecart-Salmon Brut Rosé Champagne N.V.,Normal,21473,4.4,111873,21473,4.4,...,bottle,bottles,750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[{'id': 11998745, 'amount': 59.134590327585585...",1487195,veuve-clicquot-brut-rose-champagne-nv,Veuve Clicquot Brut Rosé Champagne N.V.,Normal,20821,4.3,94047,20821,4.3,...,bottle,bottles,750,//images.vivino.com/wineries/backgrounds/JFSxY...,//thumbs.vivino.com/winery_backgrounds/JFSxYwq...,//thumbs.vivino.com/winery_backgrounds/JFSxYwq...,//thumbs.vivino.com/winery_backgrounds/JFSxYwq...,NaN,NaN,NaN
4,"[{'id': 21521100, 'amount': 75.1710893994732, ...",1488742,ruinart-brut-rose-champagne-nv,Ruinart Brut Rosé Champagne N.V.,Normal,19449,4.3,125650,19449,4.3,...,bottle,bottles,750,//images.vivino.com/wineries/backgrounds/3KO1O...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,NaN,NaN,NaN


In [19]:
df1.columns.tolist()

prices
vintage.id
vintage.seo_name
vintage.name
vintage.statistics.status
vintage.statistics.ratings_count
vintage.statistics.ratings_average
vintage.statistics.labels_count
vintage.statistics.wine_ratings_count
vintage.statistics.wine_ratings_average
vintage.statistics.wine_status
vintage.image.location
vintage.image.variations.bottle_large
vintage.image.variations.bottle_medium
vintage.image.variations.bottle_medium_square
vintage.image.variations.bottle_small
vintage.image.variations.bottle_small_square
vintage.image.variations.label
vintage.image.variations.label_large
vintage.image.variations.label_medium
vintage.image.variations.label_medium_square
vintage.image.variations.label_small_square
vintage.image.variations.large
vintage.image.variations.medium
vintage.image.variations.medium_square
vintage.image.variations.small_square
vintage.wine.id
vintage.wine.name
vintage.wine.seo_name
vintage.wine.type_id
vintage.wine.vintage_type
vintage.wine.is_natural
vintage.wine.region.id
vin

In [24]:
df1.iloc[:5, :20]

,prices,vintage.id,vintage.seo_name,vintage.name,vintage.statistics.status,vintage.statistics.ratings_count,vintage.statistics.ratings_average,vintage.statistics.labels_count,vintage.statistics.wine_ratings_count,vintage.statistics.wine_ratings_average,vintage.statistics.wine_status,vintage.image.location,vintage.image.variations.bottle_large,vintage.image.variations.bottle_medium,vintage.image.variations.bottle_medium_square,vintage.image.variations.bottle_small,vintage.image.variations.bottle_small_square,vintage.image.variations.label,vintage.image.variations.label_large,vintage.image.variations.label_medium
0,"[{'id': 25594303, 'amount': 70.10956937990868,...",1469472,laurent-perrier-brut-cuvee-champagne-rose-nv,Laurent-Perrier Brut Cuvée Champagne Rosé N.V.,Normal,35542,4.4,262835,35542,4.4,Normal,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...
1,"[{'id': 21533596, 'amount': 52.01839386443545,...",1782659,ruinart-brut-r-de-ruinart-champagne-nv,Ruinart Brut (R de Ruinart) Champagne N.V.,Normal,22102,4.3,96245,22102,4.3,Normal,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...
2,"[{'id': 8980533, 'amount': 65.84987431393853, ...",164942610,champagne-billecart-salmon-brut-rose-champagne-nv,Billecart-Salmon Brut Rosé Champagne N.V.,Normal,21473,4.4,111873,21473,4.4,Normal,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...
3,"[{'id': 11998745, 'amount': 59.134590327585585...",1487195,veuve-clicquot-brut-rose-champagne-nv,Veuve Clicquot Brut Rosé Champagne N.V.,Normal,20821,4.3,94047,20821,4.3,Normal,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...
4,"[{'id': 21521100, 'amount': 75.1710893994732, ...",1488742,ruinart-brut-rose-champagne-nv,Ruinart Brut Rosé Champagne N.V.,Normal,19447,4.3,125648,19447,4.3,Normal,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...


In [25]:
df1.iloc[:5, 21:40]

,vintage.image.variations.label_small_square,vintage.image.variations.large,vintage.image.variations.medium,vintage.image.variations.medium_square,vintage.image.variations.small_square,vintage.wine.id,vintage.wine.name,vintage.wine.seo_name,vintage.wine.type_id,vintage.wine.vintage_type,vintage.wine.is_natural,vintage.wine.region.id,vintage.wine.region.name,vintage.wine.region.name_en,vintage.wine.region.seo_name,vintage.wine.region.country.code,vintage.wine.region.country.name,vintage.wine.region.country.native_name,vintage.wine.region.country.seo_name
0,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,//images.vivino.com/thumbs/vx32r_0PT0GQIAcjiUL...,8305,Brut Cuvée Champagne Rosé,brut-cuvee-champagne-rose,3,1,False,409,Champagne,,champagne,fr,France,France,france
1,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,//images.vivino.com/thumbs/OtiVflrBQWiLs0TGB3v...,1382045,Brut (R de Ruinart) Champagne,brut-r-de-ruinart-champagne,3,1,False,409,Champagne,,champagne,fr,France,France,france
2,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,//images.vivino.com/thumbs/MbWko9RgT5eC7w-rzG_...,1211816,Brut Rosé Champagne,brut-rose-champagne,3,1,False,409,Champagne,,champagne,fr,France,France,france
3,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,//images.vivino.com/thumbs/UdM573U6TZ6mYSch-4Q...,23051,Brut Rosé Champagne,brut-rose-champagne,3,1,False,409,Champagne,,champagne,fr,France,France,france
4,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,//images.vivino.com/thumbs/Eg22whhBTpi3ri01QV1...,1148298,Brut Rosé Champagne,brut-rose-champagne,3,1,False,409,Champagne,,champagne,fr,France,France,france


In [26]:
df1.iloc[:5, 41:60]

,vintage.wine.region.country.currency.name,vintage.wine.region.country.currency.prefix,vintage.wine.region.country.currency.suffix,vintage.wine.region.country.regions_count,vintage.wine.region.country.users_count,vintage.wine.region.country.wines_count,vintage.wine.region.country.wineries_count,vintage.wine.region.country.most_used_grapes,vintage.wine.region.background_image.location,vintage.wine.region.background_image.variations.large,vintage.wine.region.background_image.variations.medium,vintage.wine.winery.id,vintage.wine.winery.name,vintage.wine.winery.seo_name,vintage.wine.winery.status,vintage.wine.winery.background_image,vintage.wine.taste.structure.acidity,vintage.wine.taste.structure.fizziness,vintage.wine.taste.structure.intensity
0,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,1572,Laurent-Perrier,laurent-perrier,0,NaN,4.261264,4.283836,3.858967
1,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,2471,Ruinart,ruinart,0,NaN,4.207951,4.249027,3.770245
2,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,1518,Billecart-Salmon,champagne-billecart-salmon,0,NaN,4.220401,4.150437,3.650437
3,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,7979,Veuve Clicquot,veuve-clicquot,0,NaN,4.314180,4.346775,3.831464
4,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",//images.vivino.com/regions/backgrounds/pDAYNE...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,//thumbs.vivino.com/region_backgrounds/pDAYNER...,2471,Ruinart,ruinart,0,NaN,4.169445,4.255943,3.763080


In [27]:
df1.iloc[:5, 61:80]

,vintage.wine.taste.structure.tannin,vintage.wine.taste.structure.user_structure_count,vintage.wine.taste.structure.calculated_structure_count,vintage.wine.taste.flavor,vintage.wine.statistics.status,vintage.wine.statistics.ratings_count,vintage.wine.statistics.ratings_average,vintage.wine.statistics.labels_count,vintage.wine.statistics.vintages_count,vintage.wine.style.id,vintage.wine.style.seo_name,vintage.wine.style.regional_name,vintage.wine.style.varietal_name,vintage.wine.style.name,vintage.wine.style.image,vintage.wine.style.background_image.location,vintage.wine.style.background_image.variations.small,vintage.wine.style.description,vintage.wine.style.blurb
0,NaN,3268.0,966.0,"[{'group': 'red_fruit', 'stats': {'count': 147...",Normal,35542,4.4,296796,100,50,french-champagne,French,Champagne,French Champagne,None,//images.vivino.com/backgrounds/styles/EDYJVTj...,//images.vivino.com/backgrounds/styles/thumbs/...,While there are many sparkling wine regions ar...,Premium sparkling wine
1,NaN,2285.0,460.0,"[{'group': 'tree_fruit', 'stats': {'count': 75...",Normal,22102,4.3,116470,93,50,french-champagne,French,Champagne,French Champagne,None,//images.vivino.com/backgrounds/styles/EDYJVTj...,//images.vivino.com/backgrounds/styles/thumbs/...,While there are many sparkling wine regions ar...,Premium sparkling wine
2,NaN,2405.0,518.0,"[{'group': 'red_fruit', 'stats': {'count': 131...",Normal,21473,4.4,111873,1,50,french-champagne,French,Champagne,French Champagne,None,//images.vivino.com/backgrounds/styles/EDYJVTj...,//images.vivino.com/backgrounds/styles/thumbs/...,While there are many sparkling wine regions ar...,Premium sparkling wine
3,NaN,1694.0,611.0,"[{'group': 'red_fruit', 'stats': {'count': 104...",Normal,20821,4.3,108263,81,50,french-champagne,French,Champagne,French Champagne,None,//images.vivino.com/backgrounds/styles/EDYJVTj...,//images.vivino.com/backgrounds/styles/thumbs/...,While there are many sparkling wine regions ar...,Premium sparkling wine
4,NaN,2123.0,418.0,"[{'group': 'red_fruit', 'stats': {'count': 835...",Normal,19447,4.3,130675,73,50,french-champagne,French,Champagne,French Champagne,None,//images.vivino.com/backgrounds/styles/EDYJVTj...,//images.vivino.com/backgrounds/styles/thumbs/...,While there are many sparkling wine regions ar...,Premium sparkling wine


In [28]:
df1.iloc[:5, 81:100]

,vintage.wine.style.body,vintage.wine.style.body_description,vintage.wine.style.acidity,vintage.wine.style.acidity_description,vintage.wine.style.country.code,vintage.wine.style.country.name,vintage.wine.style.country.native_name,vintage.wine.style.country.seo_name,vintage.wine.style.country.currency.code,vintage.wine.style.country.currency.name,vintage.wine.style.country.currency.prefix,vintage.wine.style.country.currency.suffix,vintage.wine.style.country.regions_count,vintage.wine.style.country.users_count,vintage.wine.style.country.wines_count,vintage.wine.style.country.wineries_count,vintage.wine.style.country.most_used_grapes,vintage.wine.style.wine_type_id,vintage.wine.style.food
0,3,Medium-bodied,3,High,fr,France,France,france,EUR,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",3,"[{'id': 10, 'name': 'Pork', 'background_image'..."
1,3,Medium-bodied,3,High,fr,France,France,france,EUR,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",3,"[{'id': 10, 'name': 'Pork', 'background_image'..."
2,3,Medium-bodied,3,High,fr,France,France,france,EUR,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",3,"[{'id': 10, 'name': 'Pork', 'background_image'..."
3,3,Medium-bodied,3,High,fr,France,France,france,EUR,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",3,"[{'id': 10, 'name': 'Pork', 'background_image'..."
4,3,Medium-bodied,3,High,fr,France,France,france,EUR,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",3,"[{'id': 10, 'name': 'Pork', 'background_image'..."


In [29]:
df1.iloc[:5, 101:120]

,vintage.wine.style.region.id,vintage.wine.style.region.name,vintage.wine.style.region.name_en,vintage.wine.style.region.seo_name,vintage.wine.style.region.country.code,vintage.wine.style.region.country.name,vintage.wine.style.region.country.native_name,vintage.wine.style.region.country.seo_name,vintage.wine.style.region.country.currency.code,vintage.wine.style.region.country.currency.name,vintage.wine.style.region.country.currency.prefix,vintage.wine.style.region.country.currency.suffix,vintage.wine.style.region.country.regions_count,vintage.wine.style.region.country.users_count,vintage.wine.style.region.country.wines_count,vintage.wine.style.region.country.wineries_count,vintage.wine.style.region.country.most_used_grapes,vintage.wine.style.region.parent_id,vintage.wine.style.region.background_image.location
0,409,Champagne,,champagne,fr,France,France,france,EUR,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,//images.vivino.com/regions/backgrounds/pDAYNE...
1,409,Champagne,,champagne,fr,France,France,france,EUR,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,//images.vivino.com/regions/backgrounds/pDAYNE...
2,409,Champagne,,champagne,fr,France,France,france,EUR,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,//images.vivino.com/regions/backgrounds/pDAYNE...
3,409,Champagne,,champagne,fr,France,France,france,EUR,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,//images.vivino.com/regions/backgrounds/pDAYNE...
4,409,Champagne,,champagne,fr,France,France,france,EUR,Euros,€,None,1304,5481327,565111,67550,"[{'id': 14, 'name': 'Pinot Noir', 'seo_name': ...",NaN,//images.vivino.com/regions/backgrounds/pDAYNE...


In [30]:
df1.iloc[:5, 121:140]

,vintage.wine.style.region.background_image.variations.medium,vintage.wine.style.region.statistics.wineries_count,vintage.wine.style.region.statistics.wines_count,vintage.wine.style.region.statistics.sub_regions_count,vintage.wine.style.region.statistics.parent_regions_count,vintage.wine.has_valid_ratings,vintage.top_list_rankings,vintage.year,vintage.grapes,vintage.has_valid_ratings,price.id,price.amount,price.discounted_from,price.discount_percent,price.type,price.sku,price.url,price.visibility,price.bottle_type_id
0,//thumbs.vivino.com/region_backgrounds/pDAYNER...,6400,28661,8,0,True,"[{'rank': 17, 'previous_rank': 5, 'description...",N.V.,None,True,25594303,70.109569,None,None,vc,,https://www.lewineclub.com/produit_champagne-l...,1,1
1,//thumbs.vivino.com/region_backgrounds/pDAYNER...,6400,28661,8,0,True,"[{'rank': 17, 'previous_rank': 2, 'description...",N.V.,None,True,21533596,52.018394,None,None,vc,421,https://optimuswine.com/achat-champagnes/421-r...,1,1
2,//thumbs.vivino.com/region_backgrounds/pDAYNER...,6400,28661,8,0,True,NaN,N.V.,None,True,8980533,65.849874,None,None,vc,ebc2b5fa-0fdb-400a-b03f-4332127928ec,https://www.cave-bruant.fr/champagne/4111-bill...,1,1
3,//thumbs.vivino.com/region_backgrounds/pDAYNER...,6400,28661,8,0,True,NaN,N.V.,None,True,11998745,59.134590,None,None,vc,8126,https://www.lacavedelill.com/7086/domaine-veuv...,1,1
4,//thumbs.vivino.com/region_backgrounds/pDAYNER...,6400,28661,8,0,True,NaN,N.V.,None,True,21521100,75.171089,None,None,vc,429,https://optimuswine.com/accueil/429-ruinart-br...,1,1


In [31]:
df1.iloc[:5, 141:]

,price.currency.name,price.currency.prefix,price.currency.suffix,price.bottle_type.id,price.bottle_type.name,price.bottle_type.short_name,price.bottle_type.short_name_plural,price.bottle_type.volume_ml,vintage.wine.winery.background_image.location,vintage.wine.winery.background_image.variations.large,vintage.wine.winery.background_image.variations.medium,vintage.wine.winery.background_image.variations.small,vintage.wine.taste.structure,vintage.wine.style.background_image,vintage.wine.region.background_image
0,US Dollars,$,None,1,Bottle (0.75l),bottle,bottles,750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,US Dollars,$,None,1,Bottle (0.75l),bottle,bottles,750,//images.vivino.com/wineries/backgrounds/3KO1O...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,NaN,NaN,NaN
2,US Dollars,$,None,1,Bottle (0.75l),bottle,bottles,750,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,US Dollars,$,None,1,Bottle (0.75l),bottle,bottles,750,//images.vivino.com/wineries/backgrounds/JFSxY...,//thumbs.vivino.com/winery_backgrounds/JFSxYwq...,//thumbs.vivino.com/winery_backgrounds/JFSxYwq...,//thumbs.vivino.com/winery_backgrounds/JFSxYwq...,NaN,NaN,NaN
4,US Dollars,$,None,1,Bottle (0.75l),bottle,bottles,750,//images.vivino.com/wineries/backgrounds/3KO1O...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,//thumbs.vivino.com/winery_backgrounds/3KO1O2u...,NaN,NaN,NaN


In [66]:
def get_wine_data(wine_id, year, page):
    headers = {
        "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0",
    }

    api_url = "https://www.vivino.com/api/wines/{id}/reviews?per_page=50&year={year}&page={page}"  # <-- increased the number of reviews to 9999

    data = requests.get(
        api_url.format(id=wine_id, year=year, page=page), headers=headers
    ).json()

    return data

In [72]:
df3 = pd.json_normalize(get_wine_data(68824, 2018, 1)['reviews'])
df3.head()

,id,rating,note,language,created_at,aggregated,flavor_word_matches,tagged_note,user.id,user.seo_name,...,vintage.image.variations.bottle_large,vintage.image.variations.bottle_medium,vintage.image.variations.bottle_medium_square,vintage.image.variations.bottle_small,vintage.image.variations.bottle_small_square,vintage.image.variations.label,vintage.image.variations.label_large,vintage.image.variations.label_medium,vintage.image.variations.label_medium_square,vintage.image.variations.label_small_square
0,232388428,5.0,"In my opinion, the king of Malbecs! One of the...",en,2022-01-19T00:07:09.000Z,True,"[{'id': 292, 'match': 'oak'}, {'id': 434, 'mat...","In my opinion, the king of Malbecs! One of the...",3250807,eduardo.dane,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,232474896,5.0,"Deep purple color, not clear, plenty of legs, ...",en,2022-01-20T07:27:10.000Z,True,"[{'id': 249, 'match': 'licorice'}, {'id': 334,...","Deep purple color, not clear, plenty of legs, ...",13852854,carlos-infant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,238999850,5.0,Absolutely magical Malbec. I was told this was...,en,2022-03-27T00:43:02.000Z,True,"[{'id': 242, 'match': 'leather'}, {'id': 292, ...",Absolutely magical Malbec. I was told this was...,41509040,andrew-lupton,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,246347540,5.0,6/11/22 - Fantastic and incredibly smooth. Wou...,en,2022-06-12T02:31:34.000Z,True,NaN,6/11/22 - Fantastic and incredibly smooth. Wou...,37325513,ya-sheng.lin,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,255284039,5.0,Wine + friends (family) = great combo for a Sa...,en,2022-09-19T11:24:35.000Z,True,NaN,Wine + friends (family) = great combo for a Sa...,45134664,carl.gamboa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
df3.iloc[:5, :20]

,id,rating,note,language,created_at,aggregated,flavor_word_matches,tagged_note,user.id,user.seo_name,user.alias,user.is_featured,user.visibility,user.image.location,user.image.variations.large,user.image.variations.small_square,user.statistics.followers_count,user.statistics.followings_count,user.statistics.ratings_count,user.statistics.ratings_sum
0,232388428,5.0,"In my opinion, the king of Malbecs! One of the...",en,2022-01-19T00:07:09.000Z,True,"[{'id': 292, 'match': 'oak'}, {'id': 434, 'mat...","In my opinion, the king of Malbecs! One of the...",3250807,eduardo.dane,Eduardo Danete,False,all,//images.vivino.com/avatars/001xoc7dxhc0t612ab...,//thumbs.vivino.com/avatars/001xoc7dxhc0t612ab...,//thumbs.vivino.com/avatars/001xoc7dxhc0t612ab...,53,42,288,1064.5
1,232474896,5.0,"Deep purple color, not clear, plenty of legs, ...",en,2022-01-20T07:27:10.000Z,True,"[{'id': 249, 'match': 'licorice'}, {'id': 334,...","Deep purple color, not clear, plenty of legs, ...",13852854,carlos-infant,Carlos Infante,False,all,//images.vivino.com/avatars/0088wxi7h2evvf4fa8...,//thumbs.vivino.com/avatars/0088wxi7h2evvf4fa8...,//thumbs.vivino.com/avatars/0088wxi7h2evvf4fa8...,32,24,1509,6836.7
2,238999850,5.0,Absolutely magical Malbec. I was told this was...,en,2022-03-27T00:43:02.000Z,True,"[{'id': 242, 'match': 'leather'}, {'id': 292, ...",Absolutely magical Malbec. I was told this was...,41509040,andrew-lupton,Andrew Lupton,False,all,//images.vivino.com/avatars/OOn-rZJTTD-dy0yGSQ...,//thumbs.vivino.com/avatars/OOn-rZJTTD-dy0yGSQ...,//thumbs.vivino.com/avatars/OOn-rZJTTD-dy0yGSQ...,5,5,189,661.9
3,246347540,5.0,6/11/22 - Fantastic and incredibly smooth. Wou...,en,2022-06-12T02:31:34.000Z,True,NaN,6/11/22 - Fantastic and incredibly smooth. Wou...,37325513,ya-sheng.lin,Ya Sheng Lin,False,all,//images.vivino.com/avatars/UKXv3jElQxqrFJHZwd...,//thumbs.vivino.com/avatars/UKXv3jElQxqrFJHZwd...,//thumbs.vivino.com/avatars/UKXv3jElQxqrFJHZwd...,6,3,477,2070.6
4,255284039,5.0,Wine + friends (family) = great combo for a Sa...,en,2022-09-19T11:24:35.000Z,True,NaN,Wine + friends (family) = great combo for a Sa...,45134664,carl.gamboa,Carl Gamboa,False,all,//images.vivino.com/avatars/Tb8jbydNSzCuRSAmrO...,//thumbs.vivino.com/avatars/Tb8jbydNSzCuRSAmrO...,//thumbs.vivino.com/avatars/Tb8jbydNSzCuRSAmrO...,13,14,470,1945.0


In [88]:
df3.iloc[:5, 21:40]

,user.statistics.purchase_order_count,user.language,user.background_image.location,user.background_image.variations.large,user.background_image.variations.medium,user.background_image.variations.small,vintage.id,vintage.seo_name,vintage.name,vintage.statistics.status,vintage.statistics.ratings_count,vintage.statistics.ratings_average,vintage.statistics.labels_count,vintage.statistics.reviews_count,vintage.organic_certification_id,vintage.certified_biodynamic,vintage.image.location,vintage.image.variations.large,vintage.image.variations.medium
0,0,en,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,156202841,angelica-zapata-malbec-alta-2018,Angélica Zapata Malbec Alta 2018,Normal,5014,4.6,45530,1304,None,None,//images.vivino.com/labels/DoACjN-SSSSuSfCMh3L...,//images.vivino.com/thumbs/DoACjN-SSSSuSfCMh3L...,//images.vivino.com/thumbs/DoACjN-SSSSuSfCMh3L...
1,45,en,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,156202841,angelica-zapata-malbec-alta-2018,Angélica Zapata Malbec Alta 2018,Normal,5014,4.6,45530,1304,None,None,//images.vivino.com/labels/M3Gx5_uBQROqYgolVki...,//images.vivino.com/thumbs/M3Gx5_uBQROqYgolVki...,//images.vivino.com/thumbs/M3Gx5_uBQROqYgolVki...
2,0,en,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,156202841,angelica-zapata-malbec-alta-2018,Angélica Zapata Malbec Alta 2018,Normal,5014,4.6,45530,1304,None,None,//images.vivino.com/labels/a6L0PbTQTdWIplADhD7...,//images.vivino.com/thumbs/a6L0PbTQTdWIplADhD7...,//images.vivino.com/thumbs/a6L0PbTQTdWIplADhD7...
3,0,en,//images.vivino.com/users/backgrounds/gnEJoWa3...,//thumbs.vivino.com/backgrounds/gnEJoWa3S-mNtk...,//thumbs.vivino.com/backgrounds/gnEJoWa3S-mNtk...,//thumbs.vivino.com/backgrounds/gnEJoWa3S-mNtk...,156202841,angelica-zapata-malbec-alta-2018,Angélica Zapata Malbec Alta 2018,Normal,5014,4.6,45530,1304,None,None,//images.vivino.com/labels/sq7SCO9tQEun-ZoWHdQ...,//images.vivino.com/thumbs/sq7SCO9tQEun-ZoWHdQ...,//images.vivino.com/thumbs/sq7SCO9tQEun-ZoWHdQ...
4,0,en,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,//images.vivino.com/users/backgrounds/default_...,156202841,angelica-zapata-malbec-alta-2018,Angélica Zapata Malbec Alta 2018,Normal,5014,4.6,45530,1304,None,None,//images.vivino.com/labels/xOhmj2M2RyyiZx553l6...,//images.vivino.com/thumbs/xOhmj2M2RyyiZx553l6...,//images.vivino.com/thumbs/xOhmj2M2RyyiZx553l6...


In [89]:
df3.iloc[:5, 41:60]

,vintage.image.variations.small_square,vintage.wine.id,vintage.wine.name,vintage.wine.seo_name,vintage.wine.type_id,vintage.wine.vintage_type,vintage.wine.is_natural,vintage.wine.region.id,vintage.wine.region.name,vintage.wine.region.name_en,vintage.wine.region.seo_name,vintage.wine.region.country.code,vintage.wine.region.country.name,vintage.wine.region.country.native_name,vintage.wine.region.country.seo_name,vintage.wine.region.country.currency.code,vintage.wine.region.country.currency.name,vintage.wine.region.country.currency.prefix,vintage.wine.region.country.currency.suffix
0,//images.vivino.com/thumbs/DoACjN-SSSSuSfCMh3L...,68824,Malbec Alta,malbec-alta,1,0,False,454,Mendoza,,mendoza,ar,Argentina,Argentina,argentina,ARS,Argentinean Peso,$,None
1,//images.vivino.com/thumbs/M3Gx5_uBQROqYgolVki...,68824,Malbec Alta,malbec-alta,1,0,False,454,Mendoza,,mendoza,ar,Argentina,Argentina,argentina,ARS,Argentinean Peso,$,None
2,//images.vivino.com/thumbs/a6L0PbTQTdWIplADhD7...,68824,Malbec Alta,malbec-alta,1,0,False,454,Mendoza,,mendoza,ar,Argentina,Argentina,argentina,ARS,Argentinean Peso,$,None
3,//images.vivino.com/thumbs/sq7SCO9tQEun-ZoWHdQ...,68824,Malbec Alta,malbec-alta,1,0,False,454,Mendoza,,mendoza,ar,Argentina,Argentina,argentina,ARS,Argentinean Peso,$,None
4,//images.vivino.com/thumbs/xOhmj2M2RyyiZx553l6...,68824,Malbec Alta,malbec-alta,1,0,False,454,Mendoza,,mendoza,ar,Argentina,Argentina,argentina,ARS,Argentinean Peso,$,None


In [90]:
df3.iloc[:5, 61:80]

,vintage.wine.region.country.users_count,vintage.wine.region.country.wines_count,vintage.wine.region.country.wineries_count,vintage.wine.region.country.most_used_grapes,vintage.wine.region.parent_id,vintage.wine.region.background_image.location,vintage.wine.region.background_image.variations.large,vintage.wine.region.background_image.variations.medium,vintage.wine.region.statistics.wineries_count,vintage.wine.region.statistics.wines_count,vintage.wine.region.statistics.sub_regions_count,vintage.wine.region.statistics.parent_regions_count,vintage.wine.review_status,vintage.wine.winery.id,vintage.wine.winery.name,vintage.wine.winery.seo_name,vintage.wine.winery.status,vintage.wine.winery.review_status,vintage.wine.winery.background_image
0,595653,48845,5267,"[{'id': 9, 'name': 'Malbec', 'seo_name': 'malb...",None,//images.vivino.com/regions/backgrounds/AKFL7n...,//thumbs.vivino.com/region_backgrounds/AKFL7nH...,//thumbs.vivino.com/region_backgrounds/AKFL7nH...,4068,26294,12,0,2,243347,Angélica Zapata,angelica-zapata,0,Completed,None
1,595653,48845,5267,"[{'id': 9, 'name': 'Malbec', 'seo_name': 'malb...",None,//images.vivino.com/regions/backgrounds/AKFL7n...,//thumbs.vivino.com/region_backgrounds/AKFL7nH...,//thumbs.vivino.com/region_backgrounds/AKFL7nH...,4068,26294,12,0,2,243347,Angélica Zapata,angelica-zapata,0,Completed,None
2,595653,48845,5267,"[{'id': 9, 'name': 'Malbec', 'seo_name': 'malb...",None,//images.vivino.com/regions/backgrounds/AKFL7n...,//thumbs.vivino.com/region_backgrounds/AKFL7nH...,//thumbs.vivino.com/region_backgrounds/AKFL7nH...,4068,26294,12,0,2,243347,Angélica Zapata,angelica-zapata,0,Completed,None
3,595653,48845,5267,"[{'id': 9, 'name': 'Malbec', 'seo_name': 'malb...",None,//images.vivino.com/regions/backgrounds/AKFL7n...,//thumbs.vivino.com/region_backgrounds/AKFL7nH...,//thumbs.vivino.com/region_backgrounds/AKFL7nH...,4068,26294,12,0,2,243347,Angélica Zapata,angelica-zapata,0,Completed,None
4,595653,48845,5267,"[{'id': 9, 'name': 'Malbec', 'seo_name': 'malb...",None,//images.vivino.com/regions/backgrounds/AKFL7n...,//thumbs.vivino.com/region_backgrounds/AKFL7nH...,//thumbs.vivino.com/region_backgrounds/AKFL7nH...,4068,26294,12,0,2,243347,Angélica Zapata,angelica-zapata,0,Completed,None


In [93]:
df3.iloc[:5, 81:]

,vintage.wine.winery.statistics.ratings_average,vintage.wine.winery.statistics.labels_count,vintage.wine.winery.statistics.wines_count,vintage.wine.style,vintage.wine.has_valid_ratings,vintage.year,vintage.grapes,vintage.has_valid_ratings,activity.id,activity.statistics.likes_count,...,vintage.image.variations.bottle_large,vintage.image.variations.bottle_medium,vintage.image.variations.bottle_medium_square,vintage.image.variations.bottle_small,vintage.image.variations.bottle_small_square,vintage.image.variations.label,vintage.image.variations.label_large,vintage.image.variations.label_medium,vintage.image.variations.label_medium_square,vintage.image.variations.label_small_square
0,4.4,943211,7,None,False,2018,None,True,613933549,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.4,943211,7,None,False,2018,None,True,614036287,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.4,943211,7,None,False,2018,None,True,628842640,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.4,943211,7,None,False,2018,None,True,645500284,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4.4,943211,7,None,False,2018,None,True,666042318,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
